1. 이미지 분류 모델을 만들고, 전체 코드와 분류 결과를 주피터 노트북 파일로 깃헙에 제출.
    - 이미지 데이터를 적절히 전처리 후 데이터 파이프라인 설계
    - 모델은 VGG-16(19), ResNet-34(50), GoogLeNet 등 케창딥 교재에 소개된 CNN 모델 중 하나를 선택
    - 텐서플로우 라이브러리에서 모델을 불러오지 않고 직접 모델 학습 코딩
    - Data augmentation 등 케창딥에서 배운 모든 방법을 사용해 최고 기록을 달성
2. (선택) 이미지 분류 학습 결과를 텐서보드로 시각화
    - tensorboard.dev를 사용해 텐서보드 결과를 공유할 수 있는 url 주소 링크를 자신의 깃헙 Main_Quest_2 폴더 내의 Readme.md 파일로 게시
3. (선택) 가장 성능이 좋은 모델에 CAM을 적용해, 분류 결과에 대한 근거를 시각화하고 분석
    - 케창딥의 Class Activation Map 코드를 참고해 페렴 이미지 분류기에 테스트 이미지를 넣고 추론한 결과에 CAM heatmap을 뽑아 주피터 노트북 파일에 포함해 제출

In [108]:
from pathlib import Path
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SeparableConv2D, BatchNormalization, AveragePooling2D, ZeroPadding2D, Activation, Input, Add, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [109]:
data_path = Path('C:/Users/ZAKAR/Documents/GitHub/AIFFEL/Exploration/Main_Quest_2/chest_xray')

train_data = image_dataset_from_directory(data_path / 'train', image_size=(150, 150), batch_size=32)
test_data = image_dataset_from_directory(data_path / 'test', image_size=(150, 150), batch_size=32)
val_data = image_dataset_from_directory(data_path / 'val', image_size=(150, 150), batch_size=32)

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


In [110]:
# 데이터와 레이블 크기 확인
for data_batch, labels_batch in train_data:
    print('Size of data batch:', data_batch.shape)
    print('Size of label batch:', labels_batch.shape)
    break

Size of data batch: (32, 150, 150, 3)
Size of label batch: (32,)


In [111]:
# 데이터 증강
train_data_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_data_aug = ImageDataGenerator(rescale=1./255)

train_generator = train_data_aug.flow_from_directory(
    os.path.join(data_path, 'train'),
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_data_aug.flow_from_directory(
    os.path.join(data_path, 'val'),
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


VGG-16, ResNet-34, GoogLeNet 세 가지 모델을 만든 후 각 모델의 성능을 비교해보자.

In [112]:
# VGG0-16 구현
def VGG16(input_shape=(150, 150, 3), classes=2):
    input = Input(input_shape)

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    x = Flatten()(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    output = Dense(classes, activation='softmax', name='predictions')(x)

    model = Model(inputs=input, outputs=output, name='VGG-16')
    return model

model_vgg = VGG16()
model_vgg.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model_vgg.summary()

Model: "VGG-16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0    

In [113]:
# ResNet-34 구현을 위한 잔차 연결
def Residual_Block(x, filters, pooling=False):
    residual = x
    x = Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = Conv2D(filters, 3, activation='relu', padding='same')(x)
    if pooling:
        x = MaxPooling2D(2, padding='same')(x)
        residual = Conv2D(filters, 1, strides=2)(residual)
    elif filters != residual.shape[-1]:
        residual = Conv2D(filters, 1)(residual)
    x = Add()([x, residual])

    return x

# ResNet-34 구현
def ResNet34(input_shape=(150, 150, 3), classes=2):
    img_input = Input(shape=input_shape)

    x = ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='valid', name='conv1')(x)
    x = BatchNormalization(axis=3, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = Residual_Block(x, filters=64)
    x = Residual_Block(x, filters=64)

    x = AveragePooling2D((7, 7), name='avg_pool')(x)

    x = Flatten()(x)
    x = Dense(classes, activation='softmax', name='fc' + str(classes))(x)

    model = Model(inputs=img_input, outputs=x, name='ResNet-34')

    return model

model_resnet = ResNet34()
model_resnet.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet.summary()

Model: "ResNet-34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 156, 156, 3)  0           ['input_19[0][0]']               
                                                                                                  
 conv1 (Conv2D)                 (None, 75, 75, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 bn_conv1 (BatchNormalization)  (None, 75, 75, 64)   256         ['conv1[0][0]']          

In [114]:
# GoogLeNet 구현을 위한 Inception 모듈 설계
def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv_5x5)

    pool_proj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu')(pool_proj)

    output = Concatenate(axis=-1)([conv_1x1, conv_3x3, conv_5x5, pool_proj])
    return output

# GoogLeNet 구현
def GoogLeNet(input_shape=(150, 150, 3), classes=2):
    input = Input(input_shape)
    
    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu')(input)
    x = MaxPooling2D((3, 3), padding='same', strides=(2, 2))(x)

    x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
    x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu')(x)
    x = MaxPooling2D((3, 3), padding='same', strides=(2, 2))(x)

    x = inception_module(x, filters_1x1=64, filters_3x3_reduce=96, filters_3x3=128, filters_5x5_reduce=16, filters_5x5=32, filters_pool_proj=32)
    x = inception_module(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=192, filters_5x5_reduce=32, filters_5x5=96, filters_pool_proj=64)
    
    x = AveragePooling2D((7, 7), strides=(1, 1))(x)
    x = Dropout(0.4)(x)
    x = Flatten()(x)
    
    output = Dense(classes, activation='softmax')(x)

    model = Model(inputs=input, outputs=output, name='GoogLeNet')
    return model

model_google = GoogLeNet()
model_google.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model_google.summary()

Model: "GoogLeNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_81 (Conv2D)             (None, 75, 75, 64)   9472        ['input_20[0][0]']               
                                                                                                  
 max_pooling2d_23 (MaxPooling2D  (None, 38, 38, 64)  0           ['conv2d_81[0][0]']              
 )                                                                                                
                                                                                          

In [115]:
# 각 모델에 대한 텐서보드 콜백 생성
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

vgg16_tensorboard_callback = TensorBoard(log_dir='./logs/vgg16', histogram_freq=1)
resnet34_tensorboard_callback = TensorBoard(log_dir='./logs/resnet34', histogram_freq=1)
googlenet_tensorboard_callback = TensorBoard(log_dir='./logs/googlenet', histogram_freq=1)

In [117]:
# 각 모델을 학습
model_vgg.fit(
    train_generator,
    steps_per_epoch=25,
    epochs=50,
    validation_data=val_generator,
    validation_steps=50,
    callbacks=[early_stopping, model_checkpoint, vgg16_tensorboard_callback]
)

model_resnet.fit(
    train_generator,
    steps_per_epoch=25,
    epochs=50,
    validation_data=val_generator,
    validation_steps=50,
    callbacks=[early_stopping, model_checkpoint, resnet34_tensorboard_callback]
)

model_google.fit(
    train_generator,
    steps_per_epoch=25,
    epochs=50,
    validation_data=val_generator,
    validation_steps=50,
    callbacks=[early_stopping, model_checkpoint, googlenet_tensorboard_callback]
)

Epoch 1/50
25/25 [==============================] - 12s 456ms/step - loss: 0.5795 - accuracy: 0.7350
Epoch 2/50
25/25 [==============================] - 11s 429ms/step - loss: 0.5651 - accuracy: 0.7487
Epoch 3/50
25/25 [==============================] - 11s 440ms/step - loss: 0.5716 - accuracy: 0.7450
Epoch 4/50
25/25 [==============================] - 11s 441ms/step - loss: 0.5659 - accuracy: 0.7475
Epoch 5/50
25/25 [==============================] - 12s 449ms/step - loss: 0.5777 - accuracy: 0.7400
Epoch 6/50
25/25 [==============================] - 12s 458ms/step - loss: 0.5455 - accuracy: 0.7688
Epoch 7/50
25/25 [==============================] - 12s 494ms/step - loss: 0.5665 - accuracy: 0.7513
Epoch 8/50
25/25 [==============================] - 12s 487ms/step - loss: 0.5457 - accuracy: 0.7663
Epoch 9/50
25/25 [==============================] - 11s 449ms/step - loss: 0.5908 - accuracy: 0.7237
Epoch 10/50
25/25 [==============================] - 12s 457ms/step - loss: 0.5943 - accura